### Reference

* http://pytorch.org/docs/0.3.0/
* https://github.com/yunjey/pytorch-tutorial

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets

torch.manual_seed(1)
import matplotlib.pyplot as plt
%matplotlib inline

### 텐서 

In [14]:
x = [1.,2.,3.] # 리스트
print(torch.Tensor(x))

x = [[1.,2.],[3.,4.]] # 행렬 # ndarray

print(torch.Tensor(x))

# 넘파이
T_data = np.array([[[1.,2.], [3.,4.]], # 3차원 텐서
          [[5.,6.], [7.,8.]]])
T = torch.Tensor(T_data)
print(T)


 1
 2
 3
[torch.FloatTensor of size 3]


 1  2
 3  4
[torch.FloatTensor of size 2x2]


(0 ,.,.) = 
  1  2
  3  4

(1 ,.,.) = 
  5  6
  7  8
[torch.FloatTensor of size 2x2x2]



### Autograd = 자동미분? 

In [15]:
# 넘파이
T_data = np.array([[[1.,2.], [3.,4.]], # 3차원 텐서
          [[5.,6.], [7.,8.]]])
T = torch.Tensor(T_data)
print(T)

print(Variable(T)) # 미분을 자동으로 해준다


(0 ,.,.) = 
  1  2
  3  4

(1 ,.,.) = 
  5  6
  7  8
[torch.FloatTensor of size 2x2x2]

Variable containing:
(0 ,.,.) = 
  1  2
  3  4

(1 ,.,.) = 
  5  6
  7  8
[torch.FloatTensor of size 2x2x2]



In [29]:
x = Variable( torch.Tensor([1., 2., 3]), requires_grad=True ) 
y = Variable( torch.Tensor([4., 5., 6]), requires_grad=True )
z = x + y

print(z.data)

# BUT z knows something extra.
print(z.grad_fn)


 5
 7
 9
[torch.FloatTensor of size 3]



In [30]:
# Lets sum up all the entries in z
s = z.sum()
print(s)
print(s.grad_fn)

Variable containing:
 21
[torch.FloatTensor of size 1]



In [31]:
s.backward() # calling .backward() on any variable will run backprop, starting from it.
print(x.grad)

Variable containing:
 1
 1
 1
[torch.FloatTensor of size 3]



### MNIST 데이터 불러오기 

In [2]:
train_dataset = vdatasets.MNIST(root='../data/',
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True,
                                           num_workers=1)

test_dataset = vdatasets.MNIST(root='../data/',
                               train=False, 
                               transform=transforms.ToTensor(),
                               download=True)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=64, 
                                           shuffle=True,
                                           num_workers=1)

### 모델링 

In [32]:
ACTIVATION_FUNCTION = F.relu # F.sigmoid F.tanh ...

In [4]:
class NN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(NN,self).__init__() # 부모 클래스의 생성자 호출(초기화)
        
        self.l1 = nn.Linear(input_size,hidden_size)
        self.l2 = nn.Linear(hidden_size,hidden_size)
        self.l3 = nn.Linear(hidden_size,output_size)
        
    def forward(self,inputs):
        inputs = ACTIVATION_FUNCTION(self.l1(inputs))
        inputs = ACTIVATION_FUNCTION(self.l2(inputs))
        return self.l3(inputs)

### 트레이닝 

In [5]:
EPOCH=3
LR=0.01
BATCH_SIZE=16

model = NN(784,1024,10)
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함하고 있음
optimizer = optim.SGD(model.parameters(), lr=LR)

In [33]:
for epoch in range(EPOCH):
    losses=[]
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = Variable(inputs).view(-1,784), Variable(targets)
        model.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

        losses.append(loss.data[0])
        if i % 100 == 0:
            print("[%d/%d] [%03d/%d] mean_loss : %.3f" % (epoch,EPOCH,i,len(train_loader),np.mean(losses)))
            losses=[]

### 테스트 

In [7]:
num_equal=0
for i, (inputs, targets) in enumerate(test_loader):
    inputs, targets = Variable(inputs).view(-1,784), Variable(targets)
    outputs = model(inputs)
    
    outputs = outputs.max(1)[1] # argmax
    num_equal += torch.eq(outputs,targets).sum().data[0]
    
    
print("Accuracy : " ,num_equal/len(test_dataset))

Accuracy :  0.9081
